In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("data/nhl.csv")
cities=pd.read_html("data/wikipedia_data.htm")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

In [2]:
nhl_df

,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League
0,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,2018,NHL
1,Tampa Bay Lightning*,82,54,23,5,113,.689,296,236,0.66,-0.07,.634,48,2018,NHL
2,Boston Bruins*,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL
3,Toronto Maple Leafs*,82,49,26,7,105,.640,277,232,0.49,-0.06,.567,42,2018,NHL
4,Florida Panthers,82,44,30,8,96,.585,248,246,-0.01,-0.04,.537,41,2018,NHL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,Los Angeles Kings*,82,46,28,8,100,.610,206,174,0.40,0.01,.549,38,2014,NHL
167,Phoenix Coyotes,82,37,30,15,89,.543,216,231,-0.16,0.03,.457,31,2014,NHL
168,Vancouver Canucks,82,36,35,11,83,.506,196,223,-0.30,0.03,.451,31,2014,NHL
169,Calgary Flames,82,35,40,7,77,.470,209,241,-0.35,0.04,.402,28,2014,NHL


In [3]:
cities

,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,New York City,20153634,Giants Jets[note 1],Yankees Mets[note 2],Knicks Nets,Rangers Islanders Devils[note 3]
1,Los Angeles,13310447,Rams Chargers[note 4],Dodgers Angels,Lakers Clippers,Kings Ducks
2,San Francisco Bay Area,6657982,49ers Raiders[note 6],Giants Athletics,Warriors,Sharks[note 7]
3,Chicago,9512999,Bears[note 8],Cubs White Sox,Bulls[note 9],Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars
5,"Washington, D.C.",6131977,Redskins,Nationals[note 10],Wizards[note 11],Capitals
6,Philadelphia,6070500,Eagles,Phillies[note 12],76ers,Flyers[note 13]
7,Boston,4794447,Patriots[note 14],Red Sox[note 15],Celtics,Bruins
8,Minneapolis–Saint Paul,3551036,Vikings,Twins,Timberwolves,Wild[note 16]
9,Denver,2853077,Broncos,Rockies,Nuggets[note 17],Avalanche[note 18]


In [4]:
# Diccionario de equipos de la NHL y sus ciudades correspondientes
nhl_teams = {
    'Tampa Bay Lightning': 'Tampa Bay Area',
    'Boston Bruins': 'Boston',
    'Toronto Maple Leafs': 'Toronto',
    'Florida Panthers': 'Miami–Fort Lauderdale',
    'Detroit Red Wings': 'Detroit',
    'Buffalo Sabres': 'Buffalo',
    'Montreal Canadiens': 'Montreal',
    'Ottawa Senators': 'Ottawa',
    'Washington Capitals': 'Washington, D.C.',
    'Pittsburgh Penguins': 'Pittsburgh',
    'Columbus Blue Jackets': 'Columbus',
    'New Jersey Devils': 'New York City',
    'Carolina Hurricanes': 'Raleigh',
    'New York Islanders': 'New York City',
    'New York Rangers': 'New York City',
    'Philadelphia Flyers': 'Philadelphia',
    'Nashville Predators': 'Nashville',
    'Winnipeg Jets': 'Winnipeg',
    'St. Louis Blues': 'St. Louis',
    'Dallas Stars': 'Dallas–Fort Worth',
    'Minnesota Wild': 'Minneapolis–Saint Paul',
    'Colorado Avalanche': 'Denver',
    'Chicago Blackhawks': 'Chicago',
    'Vegas Golden Knights': 'Las Vegas',
    'Calgary Flames': 'Calgary',
    'Edmonton Oilers': 'Edmonton',
    'San Jose Sharks': 'San Francisco Bay Area',
    'Anaheim Ducks': 'Los Angeles',
    'Vancouver Canucks': 'Vancouver',
    'Los Angeles Kings': 'Los Angeles',
    'Arizona Coyotes': 'Phoenix'
}

# Lista de población de las regiones metropolitanas
population_by_region = cities['Population (2016 est.)[8]'].tolist()

# Lista de relación de victorias y derrotas de los equipos de la NHL
win_loss_by_region = []

# Iterar a través del diccionario de equipos de la NHL y sus ciudades correspondientes
for team, city in nhl_teams.items():
    # Filtrar datos para el equipo y año 2018
    team_data = nhl_df[(nhl_df['team'] == team) & (nhl_df['year'] == 2018)]

    # Verificar si hay datos antes de acceder al primer elemento
    if not team_data.empty:
        # Convertir a números
        wins = int(team_data['W'].values[0])
        losses = int(team_data['L'].values[0])

        # Calcular el porcentaje de victorias y derrotas
        win_loss_ratio = wins / (wins + losses)

        # Añadir a la lista
        win_loss_by_region.append(win_loss_ratio)
    else:
        # Si no hay datos, asumir un ratio de victorias/derrotas de 0.5
        win_loss_by_region.append(0.5)

In [5]:
len(population_by_region)

51

In [6]:
len(win_loss_by_region)

31

In [7]:


stats.pearsonr(population_by_region, win_loss_by_region)

ValueError: x and y must have the same length.